# SCohenLab 3D BATCH Image Processing notebook (Simplified MCZ)





## IMPORTS

In [ ]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict
from typing import Optional, Union

import numpy as np
import pandas as pd
from skimage.measure import regionprops_table, regionprops

from aicssegmentation.core.pre_processing_utils import  image_smoothing_gaussian_slice_by_slice 
from skimage.measure import label

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_czi_image,
                                    export_inferred_organelle,
                                    list_image_files)

from infer_subc.core.img import *
from infer_subc.organelles import * 

%load_ext autoreload
%autoreload 2



In [ ]:
# NOTE:  these "constants" are only accurate for the testing MCZ dataset
from infer_subc.constants import (TEST_IMG_N,
                                    NUC_CH ,
                                    LYSO_CH ,
                                    MITO_CH ,
                                    GOLGI_CH ,
                                    PEROX_CH ,
                                    ER_CH ,
                                    LD_CH ,
                                    RESIDUAL_CH )              


## SETUP

In [ ]:
# this will be the example image for testing the pipeline below
test_img_n = TEST_IMG_N

# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
in_data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(in_data_path,im_type)
test_img_name = img_file_list[test_img_n]

# save output ".tiff" files here
out_data_path = data_root_path / "out"

if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

In [ ]:
img_data,meta_dict = read_czi_image(test_img_name)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


In [ ]:
scale[0]/scale[1], scale

Now get the single "optimal" slice of all our organelle channels....

## get the inferred cellmask, nuclei and cytoplasm objects

(takes < 1 sec)

Builde the segmentations in order




In [ ]:
###################
# SOMA, NUCLEI, CYTOPLASM, NUCLEUS
###################
nuclei_obj =  infer_nuclei_fromlabel(img_data,meta_dict, out_data_path)
soma_obj  = get_soma(img_data, nuclei,meta_dict, out_data_path)
cytoplasm_mask = get_cytoplasm(nuclei_obj , soma_obj , meta_dict, out_data_path)
# get overall summary stats for cellmask


In [ ]:
# names of organelles we have
organelle_names = ["lyso", "mitochondria","golgi","peroxisome","er","lipid"]

get_methods  = [get_lyso,
            get_mito,
            get_golgi,
            get_perox,
            get_ER,
            get_LD]

# load all the organelle segmentations
organelles = [meth(img_data,meta_dict, out_data_path) for meth in get_methods]

# get the intensities
organelle_channels = [LYSO_CH,MITO_CH,GOLGI_CH,PEROX_CH,ER_CH,LD_CH]

intensities = [img_data[ch] for ch in organelle_channels]


In [ ]:
organelle = 1
mask = cytoplasm_mask
segmentation = np.stack([
                            nuclei_object,
                            lyso_object,
                            mito_object,
                            golgi_object,
                            peroxi_object,
                            er_object,
                            LD_object ])

florescence = img_2D[:-1] # throw out residual


florescence.shape, segmentation.shape, mask.shape


## Visualize with `napari` 1
Visualize the first-pass segmentation and labeling with `napari`.

In [ ]:
viewer = napari.Viewer()

In [ ]:
viewer.add_image(
    nuclei_object,
    scale=scale,
    colormap='blue', 
    blending='additive'
)
viewer.add_image(
    lyso_object,
    scale=scale,
    colormap='cyan', 
    blending='additive'
)

viewer.add_image(
    mito_object,
    scale=scale,
    colormap='green', 
    blending='additive'
)

viewer.add_image(
    golgi_object,
    scale=scale,
    colormap='yellow', 
    blending='additive'
)


viewer.add_image(
    peroxi_object,
    scale=scale,
    colormap='bop orange', 
    blending='additive'
)


viewer.add_image(
    er_object,
    scale=scale,
    blending='additive')

viewer.add_image(
    LD_object,
    scale=scale,
    blending='additive')



In [ ]:
viewer.scale_bar.visible = True
from napari.utils.notebook_display import nbscreenshot

# viewer.dims.ndisplay = 3
# viewer.camera.angles = (-30, 25, 120)
nbscreenshot(viewer, canvas_only=True)

In [ ]:
viewer.close()


There may be a bug where the input images to the "infer_*" functions are modified in place and we might need to access them.  _MASKING_ seems to be the problem.  Also need to be clear about _when_ to apply the mask.

In [ ]:
viewer = napari.Viewer()


In [ ]:
cmaps = ['blue','cyan','green','yellow','bop orange','magenta','gray','gray','gray']

for i,organelle in enumerate(inferred_organelles):
    viewer.add_image(
        organelle,
        scale=scale,
        blending='additive',
        name = layer_names[i],
        colormap=cmaps[i]
    )


In [ ]:
viewer.scale_bar.visible = True

nbscreenshot(viewer, canvas_only=True)

In [ ]:
meta_dict

In [ ]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"



now build a function to loop over them all and export




In [ ]:

def batch_process_all_czi(data_root_path):

    # linearly unmixed ".czi" files are here
    data_path = data_root_path / "raw"
    im_type = ".czi"
    # get the list of all files
    img_file_list = list_image_files(data_path,im_type)
    files_generated = []
    for czi_file in img_file_list:
        out_fn = process_czi_image(czi_file)
        files_generated.append(out_fn)

    print(f"generated {len(files_generated)} ")
    return files_generated

def process_czi_image(czi_file_name):
    """wrapper for processing"""

    img_data,meta_dict = read_czi_image(czi_file_name)
    # # get some top-level info about the RAW data
    # channel_names = meta_dict['name']
    # img = meta_dict['metadata']['aicsimage']
    # scale = meta_dict['scale']
    # channel_axis = meta_dict['channel_axis']

    inferred_organelles, layer_names,optimal_Z = _infer_organelles(img_data)
    meta_dict['z_slice'] = optimal_Z
    out_file_n = _export_infer_organelles(inferred_organelles, layer_names, meta_dict, data_root_path)

    ## TODO:  collect stats... 

    return out_file_n

# chan_name = 'nuclei'
# out_path = data_root_path / "inferred_objects" 
# object_name = 'NU_object'

# NU_bioim = read_input_image( out_path/ f"{object_name}.ome.tiff"  )
# NU_object = NU_bioim.image
# NU_labels = label(NU_object)

# # calculate a filter dimension for median filtering which considers the difference in scale of Z
# z_factor = scale[0]//scale[1]
# med_filter_size = 4 #2D 
# med_filter_size_3D = (1,med_filter_size,med_filter_size)  # set the scale for a typical median filter

In [ ]:
output_tiffs = batch_process_all_czi(data_root_path)


35 files processed in 6 minutes 47 seconds!!

In [ ]:

tiff_img_data,tiff_meta_dict = read_czi_image(output_tiffs[-1])


In [ ]:
tiff_img_data.shape

img = tiff_meta_dict['metadata']['aicsimage']
img.dims

In [ ]:
from aicsimageio.writers import OmeTiffWriter
data_in = img_out
channel_names = [layer_names]
image_names = [img_name]
print(image_names)
# chan_names = meta_in['metadata']['aicsimage'].channel_names
dimension_order = ["CZYX"]

num_images = len(  [data_in.shape])
if data_in.dtype == "bool":
    data_in = data_in.astype(np.uint8)
    data_in[data_in > 0] = 255

physical_pixel_sizes = [meta_dict["metadata"]["aicsimage"].physical_pixel_sizes]
out_ome = OmeTiffWriter.build_ome(
        [data_in.shape],
        [data_in.dtype],
        channel_names=channel_names,  # type: ignore
        image_name=image_names,
        physical_pixel_sizes=physical_pixel_sizes,
        dimension_order=dimension_order,
    )